In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import numpy as np

In [2]:
case_details=pd.read_excel("C:/Users/nango/UNCTAD-ISDS-Navigator-release-of-ISDS-data-set-31July2019.xlsx")
print(case_details.head(10))

   NO.  YEAR OF INITIATION                       SHORT CASE NAME  \
0    1                2019                      Alcosa v. Kuwait   
1    2                2019             Axiata and Ncell v. Nepal   
2    3                2019                  Bahari v. Azerbaijan   
3    4                2019                       Canepa v. Spain   
4    5                2019         Dalal v. United Arab Emirates   
5    6                2019             Díaz Gaspar v. Costa Rica   
6    7                2019                Enel Fortuna v. Panama   
7    8                2019  Glencore and others v. Colombia (II)   
8    9                2019         Impresa Pizzarotti v. Morocco   
9   10                2019          Jetion and T-Hertz v. Greece   

                                      FULL CASE NAME  \
0                      Alcosa v. The State of Kuwait   
1  Axiata Investments (UK) Limited and Ncell Priv...   
2                      Mohammad Bahari v. Azerbaijan   
3  Canepa Green Energy Oppo

In [3]:
def extractor(column,list_omit = None):
    valid=[text.split(";\n") for text in case_details[column]]
    column_element=set()
    for element in valid:
        for element2 in element:
            column_element.add(element2)
    if list_omit!=None:
        for element_omit in list_omit:
            column_element.remove(element_omit)
    return list(column_element)

def binary_converter(column,suffix,list_omit = None):
    values=[]
    column_element_list=extractor(column,list_omit)
    for element3 in column_element_list:
        element_column=[]
        for element4 in case_details[column]:
            if element3 in element4:
                element_column.append(1)
            else: element_column.append(0)
        values.append(element_column)
    for index,a_column_element in enumerate(column_element_list):
        case_details[a_column_element+suffix]=values[index]
   

In [4]:
economic_sector=[text.split(";\n") for text in case_details["ECONOMIC SECTOR"]]
sector=set()
for sectorlist in economic_sector:
    for element in sectorlist:
        sector_class=re.findall(r"^\w*",element)[0]
        sector.add(sector_class)
for element in ['Data']:
    sector.remove(element)
    
sector_values=[]
sectorlist=list(sector)
for element in sectorlist:
    sector_column=[]
    for element2 in case_details["ECONOMIC SECTOR"]:
            if element in element2:
                sector_column.append(1)
            else: sector_column.append(0)
    sector_values.append(sector_column)

for index, sector in enumerate(sectorlist):
    case_details[sector +"_SECTOR"]=sector_values[index]

We choose to exclude certain levels:
- For alleged breaches, we exlude UNCTAD entries that are unclear or others, focusing on main allegations.
- For proceeding status, we exclude unknown ICSID annulment proceedings, discontinued for unknown reasons, unknown proceedings and pending proceedings to focus on established results.
- For proceeding types, we do not exclude anything
- For breaches found, we exclude other verdicts rendered to focus on the main allegations.

In [5]:
binary_converter("BREACHES ALLEGED",'_ALLEGED',["Unclear","Other"])
binary_converter("FOLLOW-ON PROCEEDING STATUS",'_PROCEEDING STATUS',['Unknown ICSID annulment proceedings','Discontinued for unknown reasons ICSID resubmission proceedings','Pending ICSID annulment proceedings','Pending Judicial review by national courts','Discontinued ICSID annulment proceedings'])
binary_converter('FOLLOW-ON PROCEEDING TYPE','_PROCEEDING TYPE')
binary_converter("BREACHES FOUND","_FOUND",["Other"])

Decision years are considered as the final year when a decision is rendered

In [6]:
decision_date=[re.findall(r"[21][09]\d\d",element) for element in case_details["DECISIONS"]]
decision_date=[max(list(map(int,element))) if len(element)!=0 else np.nan for element in decision_date]
case_details["FINAL DECISION DATE"]=decision_date

In [7]:
decision_date_fo=[re.findall(r"[21][09]\d\d",element) for element in case_details["FOLLOW-ON DECISIONS"]]
decision_date_fo=[max(list(map(int,element))) if len(element)!=0 else np.nan for element in decision_date_fo]
case_details["FINAL DECISION DATE WITH FOLLOW - ON"]=decision_date_fo

In [8]:
print(case_details["SHORT CASE NAME"].value_counts())
testing4=[text.split(" v. ") for text in case_details["SHORT CASE NAME"]]
case_details["COMPANY"]=[element[0] for element in testing4]


Vento v. Mexico                              1
Burlington v. Ecuador                        1
Solarpark v. Spain                           1
Conseil Economique v. The State of Kuwait    1
Besserglik v. Mozambique                     1
                                            ..
Houben v. Burundi                            1
Cemusa v. Mexico                             1
Gramercy v. Peru                             1
Turkcell v. Iran                             1
BG v. Argentina                              1
Name: SHORT CASE NAME, Length: 983, dtype: int64


In [9]:
print(case_details.head(5))

   NO.  YEAR OF INITIATION                SHORT CASE NAME  \
0    1                2019               Alcosa v. Kuwait   
1    2                2019      Axiata and Ncell v. Nepal   
2    3                2019           Bahari v. Azerbaijan   
3    4                2019                Canepa v. Spain   
4    5                2019  Dalal v. United Arab Emirates   

                                      FULL CASE NAME  \
0                      Alcosa v. The State of Kuwait   
1  Axiata Investments (UK) Limited and Ncell Priv...   
2                      Mohammad Bahari v. Azerbaijan   
3  Canepa Green Energy Opportunities I, S.á r.l. ...   
4  Shokat Mohammed Dalal v. United Arab Emirates ...   

                                      APPLICABLE IIA  \
0                          Kuwait - Spain BIT (2005)   
1                  Nepal - United Kingdom BIT (1993)   
2  Azerbaijan - Iran, Islamic Republic of BIT (1996)   
3                   The Energy Charter Treaty (1994)   
4   United Arab 

In [10]:
target=['ICSID(AF)',"UNCITRAL"]
case_details["ARBITRAL RULES"][(case_details["ARBITRAL RULES"]=="ICSID (International Centre for Settlement of Investment Disputes)")|(case_details["ARBITRAL RULES"]=="ICSID AF (ICSID Additional Facility)")]="ICSID(AF)"
filters=[True if case in target else False for case in case_details["ARBITRAL RULES"]]
case_details=case_details[filters]

C:\Users\nango\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
filter2=case_details['APPLICABLE IIA'].str.contains("BIT")
case_details=case_details[filter2]

In [12]:
case_details["AWARD CLAIMED IN US MILLION"]=[re.findall(r'([\d.]*)\sUSD',string) for string in case_details["AMOUNT CLAIMED (EXPRESSED IN MILLIONS)"]]
case_details["AWARD CLAIMED IN US MILLION"]=[string[0] if len(string)>0 else np.nan for string in case_details["AWARD CLAIMED IN US MILLION"]]
case_details["AWARD RECEIVED IN US MILLION"]=[re.findall(r'([\d.]*)\sUSD',string) for string in case_details["AMOUNT AWARDED (OR SETTLED FOR) (EXPRESSED IN MILLIONS)"]]
case_details["AWARD RECEIVED IN US MILLION"]=[string[0] if len(string)>0 else np.nan for string in case_details["AWARD RECEIVED IN US MILLION"]]

In [13]:
import world_bank_data as wb
countries=wb.get_countries()
current_name=["Egypt, Arab Rep.","Cote d'Ivoire","Korea, Rep.",'Lao PDR',"Korea, Dem. People’s Rep.","Vietnam","Venezuela, RB","Kyrgyz Republic","Gambia, The","Slovak Republic","Yemen, Rep.","Congo, Dem. Rep.","Iran, Islamic Rep."]
new_name=["Egypt","Ivory Coast","South Korea","Laos","North Korea","Viet Nam","Venezuela","Venezuela","Kyrgyzstan","Gambia","Slovakia","Yemen","Congo, Democratic Republic of the","Iran"]

for i in range(len(current_name)):
    countries.name[countries.name==current_name[i]]=new_name[i]

C:\Users\nango\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [14]:
case_details["RESPONDENT STATE"][case_details["RESPONDENT STATE"]=="Bolivia, Plurinational State of"]="Bolivia"
case_details["RESPONDENT STATE"][case_details["RESPONDENT STATE"]=="Korea, Republic of"]="South Korea"
case_details["RESPONDENT STATE"][case_details["RESPONDENT STATE"]=="Tanzania, United Republic of"]="Tanzania"
case_details["RESPONDENT STATE"][case_details["RESPONDENT STATE"]=="Venezuela, Bolivarian Republic of"]="Venezuela"
case_details["RESPONDENT STATE"][case_details["RESPONDENT STATE"]=="Czechia"]="Czech Republic"
case_details["RESPONDENT STATE"][case_details["RESPONDENT STATE"]=="Lao People's Democratic Republic"]="Laos"
case_details["RESPONDENT STATE"][case_details["RESPONDENT STATE"]=="Moldova, Republic of"]="Moldova"
case_details["RESPONDENT STATE"][case_details["RESPONDENT STATE"]=="United States of America"]="United States"
case_details["RESPONDENT STATE"][case_details["RESPONDENT STATE"]=="Iran, Islamic Republic of"]="Iran"

C:\Users\nango\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\nango\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\nango\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so w

In [15]:
current_name_inv=["Bolivia, Plurinational State of","Korea, Republic of","Tanzania, United Republic of","Venezuela, Bolivarian Republic of","Czechia","Lao People's Democratic Republic","Moldova, Republic of","United States of America","Iran, Islamic Republic of"]
new_name_inv=["Bolivia","South Korea","Tanzania","Venezuela","Czech Republic","Laos","Moldova","United States","Iran"]

for i in range(len(current_name_inv)):
    case_details["HOME STATE OF INVESTOR"][case_details["HOME STATE OF INVESTOR"]==current_name_inv[i]]==new_name_inv[i]

In [16]:
countries2=countries[["iso2Code","name","region","incomeLevel"]]
export=pd.merge(case_details,countries2,left_on=['RESPONDENT STATE'], right_on=['name'],how='left')
final_df=pd.merge(export,countries2,left_on=['HOME STATE OF INVESTOR'],right_on=["name"],how='left')
#x is for respondent state, y is for investor state.

In [17]:
final_df=final_df.drop(columns=["FULL CASE NAME","ADMINISTERING INSTITUTION","LINKS TO BACKGROUND SOURCES","ICSID ANNULMENT COMMITTEE MEMBERS","LINK TO ITALAW'S CASE PAGE","FOLLOW-ON INDIVIDUAL OPINIONS","ARBITRATORS","AMOUNT CLAIMED (EXPRESSED IN MILLIONS)","AMOUNT AWARDED (OR SETTLED FOR) (EXPRESSED IN MILLIONS)","FOLLOW-ON DECISIONS","FOLLOW-ON PROCEEDING TYPE","FOLLOW-ON PROCEEDING STATUS","DECISIONS","ECONOMIC SECTOR","ECONOMIC SUBSECTOR","INDIVIDUAL OPINIONS","Data not available_ALLEGED"])


In [18]:
final_df.to_excel("C:/Users/nango/UNCSTAD full data remodeled.xlsx",index=False)